In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import sklearn
import itertools
import time
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.preprocessing import scale

%matplotlib inline
plt.style.use('seaborn-v0_8-white')

In [10]:
#Read file
wireDF = pd.read_excel('Wire_bond.xlsx', usecols = [1, 2, 3, 4, 5, 6, 7])

#Set row names to correct names
wireDF.columns = wireDF.iloc[0]

#remove the first row afterwards
wireDF = wireDF[1:]

wireDF.reset_index(drop = True, inplace = True)

wireDF.rename(columns = {col: col.replace(' ', '_') for col in wireDF.columns}, inplace = True)
wireDF.rename(columns = {"wire_lenghth": "wire_length"}, inplace = True)
wireDF = wireDF.astype(float)

wireDF.info()
wireDF.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pull_strength           22 non-null     float64
 1   die_height              22 non-null     float64
 2   post_height             22 non-null     float64
 3   loop_height             22 non-null     float64
 4   wire_length             22 non-null     float64
 5   bond_width_on_the_die   22 non-null     float64
 6   bond_width_on_the_post  22 non-null     float64
dtypes: float64(7)
memory usage: 1.3 KB


,pull_strength,die_height,post_height,loop_height,wire_length,bond_width_on_the_die,bond_width_on_the_post
0,8.0,5.2,17.0,28.6,83.0,1.9,1.6
1,8.0,5.2,19.6,29.6,94.9,2.1,2.3
2,8.3,5.8,19.8,32.4,89.7,2.1,1.8
3,8.5,6.4,19.6,31.0,96.2,2.0,2.0
4,8.8,5.8,19.4,32.4,95.6,2.2,2.1


In [11]:
print("Number of null vals: ", wireDF.isnull().sum())

Number of null vals:  0
pull_strength             0
die_height                0
post_height               0
loop_height               0
wire_length               0
bond_width_on_the_die     0
bond_width_on_the_post    0
dtype: int64


In [31]:
X = wireDF.drop(['pull_strength'], axis = 1)
y = wireDF.drop(['die_height', 'post_height', 'loop_height', 'wire_length', 'bond_width_on_the_die', 'bond_width_on_the_post'], axis = 1)

In [27]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [28]:
def getBest(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [32]:
models_best = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,8):
    models_best.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

TypeError: reduction operation 'argmin' not allowed for this dtype